<h3>Estágio em Dados - Itaú Unibanco - Desafio
<h3>
Código desenvolvido por : Lucca Machado da Silva    

In [ ]:
#Utilizando a biblioteca pandas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm


import warnings
warnings.simplefilter(action='ignore', category=(FutureWarning, DeprecationWarning))

In [ ]:
#Carregando o arquivo CSV para um DataFrame
df = pd.read_csv('Ecommerce_DBS.csv')

In [ ]:
#Verificando os dados inicais do DataFrame
df

In [ ]:
#Obtendo informações gerais sobre o conjunto de dados, através disso é possível verificar que não temos missing values
#e os tipos dos dados de cada coluna
df.info()

In [ ]:
df.describe()

In [ ]:
#Verificando todas as chaves
df.keys()

In [ ]:
#A chave 'Customer Age ' possui um espaço no final, então vamos remover esse espaço para padronizar
df.rename(columns={'Customer Age ': 'Customer Age'}, inplace=True)

In [ ]:
#Confirmando que a mudaça foi realizada
df.keys()

In [ ]:
#Verificando por duplicatas, neste caso, não possuimos duplicatas para limpar
df.duplicated().sum()

# Qual os produtos mais vendidos considerando os últimos 3 anos ?

In [ ]:
df_first_answer = df.copy() #Criando uma cópia para responder a prgunta
df_first_answer['Purchase Date'] = pd.to_datetime(df_first_answer['Purchase Date'],format='%d/%m/%Y') #Transformando a data de str para datetime
df_first_answer = df_first_answer[df_first_answer['Purchase Date'] > '30/05/2021'] #Filtrando para datas maiores que 30/05/2021 (dia da resposta)
produtos_vendidos = df_first_answer.groupby('Product Category')['Quantity'].sum() #Agrupando por categoria e somando as quantities

In [ ]:
produtos_vendidos

In [ ]:
total_sales = produtos_vendidos.sum()
def autopct_with_amount(pct):
    return f"{pct:.1f}% ({int(pct/100 * total_sales):,})"  
plt.pie(produtos_vendidos, labels=produtos_vendidos.index, autopct=autopct_with_amount, startangle=140)
plt.title("Distribuição de tipos de produtos vendidos nos últimos 3 anos")
plt.show()

Sendo assim, é possível observar que os produtos mais vendidos seguem esta ordem, de maior para menor em vendas respectivamente, Roupas, Livros, Eletrônicos e Produtos para casa

# Qual o produto mais caro e o mais barato ?

In [ ]:
df_second_answer = df.copy() #Criando uma cópia para responder a pergunta

In [ ]:
#getting the most expensive 'Product Price' for each product category, the same for the cheaper them creating a df with both
most_expensive = df_second_answer.groupby('Product Category')['Product Price'].max()
cheaper = df_second_answer.groupby('Product Category')['Product Price'].min()
most_expensive = most_expensive.reset_index()
cheaper = cheaper.reset_index()
most_expensive.rename(columns={'Product Price': 'Mais caro'}, inplace=True)
cheaper.rename(columns={'Product Price': 'Mais barato'}, inplace=True)
df_second_answer = pd.merge(most_expensive, cheaper, on='Product Category')

In [ ]:
df_second_answer.plot(x='Product Category', kind='bar')
plt.title('Produto mais caro e mais barato por categoria')
plt.ylabel('Preço')
plt.xlabel('Categoria')
plt.show()

# Qual a categoria de produto mais vendida e a menos vendida? Qual categoria mais e menos cara?

In [ ]:
df_third_answer = df.copy()
#getting the amount of sales for each product category
sales = df_third_answer.groupby('Product Category')['Quantity'].sum()
#Letting just the greatest and lower value in the sales
sales = sales.reset_index()
sales = sales.sort_values(by='Quantity')
sales = sales.reset_index(drop=True)
sales = sales.iloc[[0, -1], :]

In [ ]:
sales

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
plot = sns.barplot(x='Product Category', y='Quantity', data=sales,palette='Set2')

for p in plot.patches:
    plot.annotate(format(p.get_height(), '.0f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points')
    
plt.title('Categoria de produto mais e menos vendida')
plt.ylabel('Quantidade')
plt.xlabel('Categoria')
plt.show()

In [ ]:
#Extraindo a mediana de preço de produto por cada categoria
median_prices = df_third_answer.groupby('Product Category')['Product Price'].median()

#bar plot sns
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

plot = sns.barplot(x=median_prices.index, y=median_prices,palette='Set2')

for p in plot.patches:
    plot.annotate(format(p.get_height(), '.2f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points')
    
plt.title('Mediana de preço por categoria de produto')
plt.ylabel('Preço')
plt.xlabel('Categoria')
plt.show()

# Qual o produto com melhor e pior NPS?

In [ ]:
df_forth_answer = df.copy()

mean_nps = df_forth_answer.groupby('Product Category')['NPS'].mean()


In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
plot = sns.barplot(x=mean_nps.index, y=mean_nps, palette='Set2')

for p in plot.patches:
    plot.annotate(format(p.get_height(), '.2f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points')

plt.title('NPS médio por categoria de produto')
plt.ylabel('NPS')
plt.xlabel('Categoria')
plt.show()

<h1> Resolução do desafio

Pensando no melhor tipo de solução possível, não se pode levar em consideração apenas os dados de gênero, idade, fonte e categoria de produto, pois o valor gasto também é um fator extremamente importante. Portanto, foram utilizadas as colunas 'Product Category', 'Customer Age', 'Gender', 'Source' e 'Total Purchase Amount' para responder ao desafio.

In [ ]:
colunas_a_serem_mantidas = ['Product Category','Customer Age','Gender','Source', 'Total Purchase Amount']
df = df[colunas_a_serem_mantidas]

df.rename(columns={'Total Purchase Amount': 'Total_Purchase_Amount'}, inplace=True)
df.rename(columns={'Product Category': 'Product_Category'}, inplace=True)
df.rename(columns={'Customer Age': 'Customer_Age'}, inplace=True)

## Primeira Hipótese: Não existe diferença significativa do Total_Purchase_Amount entre os gêneros

**Hipótese nula:** Não existe diferença significativa do Total_Purchase_Amount entre os gêneros.

**Hipótese alternativa:** Existe diferença significativa do Total_Purchase_Amount entre os gêneros.


In [ ]:
coluns_first_hypothesis= ['Gender', 'Total_Purchase_Amount']
first_hypothesis_df = df[coluns_first_hypothesis]
#Obtendo apenas as colunas que precisamos para a validação da hipótese

In [ ]:
first_hypothesis_df
#Exibindo o dataframe para aprimeira hipótese

In [ ]:
formula_first_hypothesis = 'Total_Purchase_Amount ~ Gender'
model_first_hypothesis  = ols(formula_first_hypothesis, data=first_hypothesis_df).fit()
#Criando um modelo linear para a primeira hipótese

In [ ]:
anova_table_first_hypothesis = sm.stats.anova_lm(model_first_hypothesis, typ=2)
#Obtendo a tabela ANOVA para a primeira hipótese

In [1768]:
print(anova_table_first_hypothesis)

                sum_sq        df         F    PR(>F)
Gender    1.122536e+06       1.0  0.539147  0.462787
Residual  5.205111e+11  249998.0       NaN       NaN


In [ ]:
df_first_hypothesis = df.groupby(['Gender'])['Total_Purchase_Amount'].median()
df_first_hypothesis = df_first_hypothesis.reset_index()

In [1767]:
df_first_hypothesis

,Gender,Total_Purchase_Amount
0,Female,2725.0
1,Male,2722.5


In [ ]:
sns.set(style="whitegrid")

# Create the bar plot
plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x="Gender", y="Total_Purchase_Amount", data=df_first_hypothesis,palette='Set2')

# Add title and labels
plt.title("Valor total da compra por gênero")
plt.xlabel("Gênero")
plt.ylabel("Valor total da compra")

plt.show()

Os dados da tabela ANOVA demonstram que o Total_Purchase_Amount não tem uma relação significante entre os gêneros, sendo assim, a hipótese nula é aceita, em que o gênero não tem uma influência significante no Total_Purchase_Amount.

## Segunda Hipótese: Não existe diferença significativa do Total_Purchase_Amount entre as diferentes categorias de produto

**Hipótese nula:** Não existe diferença significativa do Total_Purchase_Amount entre as diferentes categorias de produto.

**Hipótese alternativa:** Existe diferença significativa do Total_Purchase_Amount entre as diferentes categorias de produto.


In [ ]:
coluns_second_hypothesis= ['Product_Category', 'Total_Purchase_Amount']
second_hypothesis_df = df[coluns_second_hypothesis]
#Obtendo apenas as colunas que precisamos para a validação da hipótese

In [ ]:
formula_second_hypothesis = 'Total_Purchase_Amount ~ Product_Category'
model_second_hypothesis  = ols(formula_second_hypothesis, data=second_hypothesis_df).fit()
#Criando um modelo linear para a segunda hipótese

In [ ]:
anova_table_second_hypothesis = sm.stats.anova_lm(model_second_hypothesis, typ=2)
#Obtendo a tabela ANOVA para a segunda hipótese

In [1766]:
anova_table_second_hypothesis

,sum_sq,df,F,PR(>F)
Product_Category,1.566396e+07,3.0,2.507816,0.056962
Residual,5.204966e+11,249996.0,NaN,NaN


In [ ]:
df_second_hypothesis = df.groupby(['Product_Category'])['Total_Purchase_Amount'].median()
df_second_hypothesis = df_second_hypothesis.reset_index()

In [ ]:
df_second_hypothesis

In [ ]:
sns.set(style="whitegrid")

# Create the bar plot
plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x="Product_Category", y="Total_Purchase_Amount", data=df_second_hypothesis,palette='Set2')

# Add title and labels
plt.title("Valor total da compra por categoria de produto")
plt.xlabel("Categoria de produto")
plt.ylabel("Valor total da compra")
plt.show()

Os dados da tabela ANOVA demonstram que o Total_Purchase_Amount tem uma relação significante em relação ao Product_Category, ou seja, descartamos a hipótese nula e aceitamos a hipótese alternativa.

## Segunda Hipótese: Não existe diferença significativa do Total_Purchase_Amount entre as diferentes sources

**Hipótese nula:** Não existe diferença significativa do Total_Purchase_Amount entre as diferentes sources.

**Hipótese alternativa:** Existe diferença significativa do Total_Purchase_Amount entre as diferentes sources.

In [1761]:
coluns_third_hypothesis= ['Product_Category', 'Total_Purchase_Amount','Source','Gender']
third_hypothesis_df = df[coluns_third_hypothesis]
#Obtendo apenas as colunas que precisamos para a validação da hipótese

In [1762]:
formula_third_hypothesis = 'Total_Purchase_Amount ~ Product_Category + Source + Gender'
model_third_hypothesis  = ols(formula_third_hypothesis, data=third_hypothesis_df).fit()
#Criando um modelo linear para a segunda hipótese

In [1764]:
anova_table_third_hypothesis = sm.stats.anova_lm(model_third_hypothesis, typ=2)
#Obtendo a tabela ANOVA para a segunda hipótese

In [1765]:
anova_table_third_hypothesis

,sum_sq,df,F,PR(>F)
Product_Category,1.564018e+07,3.0,2.503980,0.057255
Source,1.374444e+06,3.0,0.220047,0.882536
Gender,1.130389e+06,1.0,0.542923,0.461225
Residual,5.204941e+11,249992.0,NaN,NaN


In [ ]:
df['Customer_Age'].unique() #Idades observadas no dataset

A próxima celula de código será responsável pela separação em 4 diferentes dataframes, separando cada categoria de produto individualmente

In [ ]:
compradores_electronics = df[df['Product_Category'] == 'Electronics'] #Filtrando os compradores de eletrônicos
compradores_electronics = compradores_electronics.drop(columns='Product_Category') #Removendo a coluna 'Product Category'
compradores_electronics.reset_index(drop=True, inplace=True) #Resetando o índice

compradores_home = df[df['Product_Category'] == 'Home']
compradores_home = compradores_home.drop(columns='Product_Category')
compradores_home.reset_index(drop=True, inplace=True)

compradores_clothing = df[df['Product_Category'] == 'Clothing']
compradores_clothing = compradores_clothing.drop(columns='Product_Category') 
compradores_clothing.reset_index(drop=True, inplace=True)

compradores_books = df[df['Product_Category'] == 'Books']
compradores_books = compradores_books.drop(columns='Product_Category')
compradores_books.reset_index(drop=True, inplace=True)

In [ ]:
#Exemplo da tabela de compradores de eletronicos após separação
compradores_electronics

In [ ]:
#esta função é responsável pela separação em faixas etárias, neste caso, de 18 até 73 anos, com intervalos de 5 anos
def separate_in_age_group(df):
    bins = list(range(18, 69, 10)) + [100]
    labels = [f"{i}-{i+9}" for i in range(18, 67, 10)] + ["68+"]
    df['age_group'] = pd.cut(df['Customer_Age'], bins=bins, labels=labels, right=False)
    df.drop(columns='Customer_Age', inplace=True) #Removendo a coluna de idade, agora desnecessária


In [ ]:
separate_in_age_group(compradores_electronics)
separate_in_age_group(compradores_home)
separate_in_age_group(compradores_clothing)
separate_in_age_group(compradores_books)

**Como demonstrado anteriormente pelo valor-p, o gênero não é relevante estatisticamente para nossa análise. Portanto, a função `agrupar_age_groups` agrupará os dados de cada tipo de produto em conjuntos com faixas etárias e fontes iguais, sem distinção de gênero.**


In [ ]:
def agrupar_age_groups(df):
    #std_price_quantity = df.groupby(['age_group', 'Gender','Source'])['Price*Quantity'].std().reset_index()
    std_price_quantity = df.groupby(['age_group','Source'])['Total_Purchase_Amount'].median().reset_index()
    source_counts = df.groupby(['age_group','Source']).size().reset_index(name='Source_Count')
    df = pd.merge(std_price_quantity, source_counts, on=['age_group','Source'])
    return df

In [ ]:
compradores_electronics_gruped = agrupar_age_groups(compradores_electronics)
compradores_home_gruped = agrupar_age_groups(compradores_home)
compradores_clothing_gruped = agrupar_age_groups(compradores_clothing)
compradores_books_gruped = agrupar_age_groups(compradores_books)

**Dessa forma, as tabelas `compradores_electronics_grouped`, `compradores_home_grouped`, `compradores_clothing_grouped` e `compradores_books_grouped` conterão os dados de compras dos diferentes tipos de produtos. As tabelas apresentarão a quantidade de compras realizadas por cada faixa etária com diferentes fontes, além da mediana do Total_Purchase_Amount gasto por esses grupos.**


In [ ]:
compradores_electronics_gruped

#### Um fator extremamente importante da minha solução é a variável "Rate". Ela é obtida através da multiplicação do desvio padrão do 'Total Purchase Amount' dos grupos (Gênero, Faixa Etária e Source) pela quantidade de compras que fazem parte do mesmo grupo.

#### O Rate serve como um indicador para mostrar o quanto aquele grupo gastou de fato com aquele determinado produto, servindo como um forte indicativo sobre qual seria o melhor grupo para vendermos determinado tipo de produto.


In [ ]:
compradores_electronics_gruped['Rate'] = compradores_electronics_gruped['Total_Purchase_Amount'] * compradores_electronics_gruped['Source_Count']
compradores_electronics_gruped.drop(columns=['Source_Count','Total_Purchase_Amount'], inplace=True)
compradores_electronics_gruped = compradores_electronics_gruped.sort_values(by='Rate', ascending=False) #Organizando o DataFrame de forma decrescente

#compradores_electronics_gruped['Rate'] = compradores_electronics_gruped['Price*Quantity'] * compradores_electronics_gruped['Source_Count']
#compradores_electronics_gruped.drop(columns=['Source_Count','Price*Quantity'], v binplace=True)
#compradores_electronics_gruped = compradores_electronics_gruped.sort_values(by='Rate', ascending=False) #Organizando o DataFrame de forma decrescente

In [ ]:
compradores_electronics_gruped

In [ ]:
formula = 'Total_Purchase_Amount ~ age_group + Gender + Source + age_group:Gender + age_group:Source + Gender:Source'

In [ ]:
model_test = ols(formula, data=compradores_electronics).fit()

In [ ]:
anova_table = sm.stats.anova_lm(model_test, typ=2)

In [ ]:
anova_table

In [ ]:
formula = 'Rate ~ age_group + Gender + Source + age_group:Gender + age_group:Source + Gender:Source'

In [ ]:
model = ols(formula, data=compradores_electronics_gruped).fit()

In [ ]:
anova_table = sm.stats.anova_lm(model, typ=2)

In [ ]:
print(anova_table) 

The ANOVA table provides the following key insights:

age_group: The F-value is 466.10 with a p-value of 
6.75×10^−16. This indicates a significant effect of "age_group" on the "Rate".

Gender: The F-value is 0.33 with a p-value of 0.573. This indicates that "Gender" does not have a significant effect on the "Rate".

Source: The F-value is 228.11 with a p-value of 
9.82×10^−13. This indicates a significant effect of "Source" on the "Rate".

age_group: The interaction between "age_group" and "Gender" has an F-value of 0.69 with a p-value of 0.637, indicating no significant interaction effect.
age_group: The interaction between "age_group" and "Source" has an F-value of 4.90 with a p-value of 0.0019, indicating a significant interaction effect.
Gender: The interaction between "Gender" and "Source" has an F-value of 0.34 with a p-value of 0.797, indicating no si

In [ ]:
#Aqui, estou utoizando o MinMaxScaler da biblioteca sklearn para normalizar os valores da coluna 'Rate'
#Neste caso, os valores de Rate irão de 0 a 1, onde 1 é o indicativo do melhor grupo para vender e 0 o pior grupo comparado a todos os outros
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
compradores_electronics_gruped[['Rate']] = scaler.fit_transform(compradores_electronics_gruped[['Rate']])
compradores_electronics_gruped = compradores_electronics_gruped.sort_values(by='Rate', ascending=False) #Organizando o DataFrame de forma decrescente

In [ ]:
compradores_electronics_gruped
#saving compradores_electronics_gruped to csv

In [ ]:
compradores_electronics_gruped

In [ ]:
df